In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
gpu = torch.device('cuda:0')

In [ ]:
import numpy as np
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt

import torchvision
from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Pad
from torchvision.io import decode_image
from torchinfo import summary as model_summary

In [ ]:
torch.manual_seed(555)

In [ ]:
def pad_to_500(img):
    down = 500 - img.shape[1]
    right = 500 - img.shape[2]
    pad = Pad((0,0,right,down))
    return pad(img)

In [ ]:
# Subset of VOC'2012 with object removed that CONTAIN ONLY 1 IMAGE
class CustomDataset(Dataset):
    def __init__(self, label_dir, img_dir, transform=None, target_transform=None):
        self.bbox_label = pd.read_csv(label_dir)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.bbox_label)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.bbox_label.iloc[idx, 0])
        image = decode_image(img_path).type(torch.float32)
        label = torch.tensor([self.bbox_label.iloc[idx, i] for i in range(1,5)]).type(torch.float32)
        if self.transform:
            image = pad_to_500(self.transform(image))
        if self.target_transform:
            label = self.target_transform(label)
        return image.to(gpu), label.to(gpu)

    def get_imgname(self, idx):
        return self.bbox_label.iloc[idx, 0]

In [ ]:
def get_imgname(revoc, img):
    for idx in range(len(revoc)):
        ds_img, _ = revoc[idx]
        if torch.equal(img, ds_img):
            return revoc.get_imgname(idx)

In [ ]:
resnet_preprocess = torchvision.transforms.Compose([
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])